In [1]:
# 모든 추정기 클래스들을 리스트 형태로 가져오는 데 사용하는 함수입니다.
from sklearn.utils import all_estimators

In [ ]:
# k겹 교차 검증에 필요한 도구들을 가져옵니다.
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

In [5]:
def model_selection(type_filter = "", X_y = (None, None, None, None)) :
  estimators = all_estimators(type_filter= type_filter)

  count = 0
  X_train = X_y[0]
  X_test = X_y[1]
  y_train = X_y[2]
  y_test = X_y[3]

  for (name, estimator) in estimators:
    # 오류가 나는 경우에 무시하도록 예외 처리 구문을 구성합니다. 
    try:  
      # 각 분류기의 인스턴스(즉, 모형)를 구성합니다.
      # 예를 들어 for 문이 첫 번째 돌 때 다음 문장의 꼴은
      # 분류기_인스턴스 = sklearn.ensemble._weight_boosting.AdaBoostClassifier() 꼴이 된다. 
      model = estimator()         
    except:
      # 클래스로 인스턴스를 생성할 때 파라미터 개수가 일치하지 않는 경우를 무시하게 합니다.
      pass

    # 훈련용 데이터를 사용해 모형을 훈련합니다.
    model.fit(X_train, y_train)   

    # 검정용 데이터를 사용해 예측치를 뽑아냅니다.  
    y_pred = model.predict(X_test)  

    # 정확도 점수를 출력합니다.
    if type_filter == "classifier" :
      indicator = Indicator_Classification(y_test, y_pred)
      accuracy_score = f"{indicator.Accuracy()}"
      precision_score = f"{indicator.Precision()}"
      f1_score = f"{indicator.F1_score()}"
      recall_score = f"{indicator.Recall()}"
      ROC_AUC = f"{indicator.ROC_AUC()}"

      print(f'[{count:2d}] {name:30s} - Accuracy : {accuracy_score}, Precision : {precision_score}, F1_score : {f1_score}, Recall : {recall_score}, ROC_AUC : {ROC_AUC}')

    elif type_filter == "regressor" :
      indicator = Indicator_Regression(y_test, y_pred)
      MAE = f"{indicator.MAE()}"
      MSE = f"{indicator.MSE()}"
      r2_score = f"{indicator.R2_score()}"
      RMSE = f"{indicator.RMSE()}"

      print(f'[{count:2d}] {name:30s} - MAE : {MAE}, MSE : {MSE}, R2_score : {r2_score}, RMSE : {RMSE}')

    count = count + 1

In [ ]:
# def Model_Selection_cv(valida_model) :
#   for(분류기명, 분류기_클래스) in 분류기_클래스들:
#   # 오류가 나는 경우에 무시하도록 예외 처리 구문을 구성합니다. 
#   try:  
#     # 각 분류기의 인스턴스(즉, 모형)를 구성합니다.
#     분류기_인스턴스 = 분류기_클래스()         
#   except:
#     # 클래스로 인스턴스를 생성할 때 파라미터 개수가 일치하지 않는 경우를 무시하게 합니다.
#     pass

#   # score라는 메서드를 지닌 인스턴스를 골라냅니다.  
#   # 왜냐하면 cross_val_score 함수에서 score 메서드를 사용하기 때문입니다.
#   # 각 추정기별로 score 메서드는 일반적으로 평균 정확도 점수를 반환해줍니다.
#   # 이에 대해서는 각 추정기 클래스의 score 메서드를 확인해 봅시다. 
#   if hasattr(분류기_인스턴스, "score"):
#     try:
#       점수들 = cross_val_score(분류기_인스턴스, 훈련용_X, 훈련용_y, cv=k겹_교차검증_모형)
#     except:
#       pass
#     print(f'[{순번:2d}] {분류기명:30s} 교차 검증 점수 = {np.round(점수들, 4)} \t 평균 점수 = {np.mean(점수들)*100:.2f}%')
#     # print(f'[{순번:2d}] {분류기명:30s} 교차 검증 점수 = {점수들} \t 평균 점수 = {np.mean(점수들)*100:.2f}%')
  
#   순번 = 순번 + 1

In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import label_binarize

import math

In [7]:
class Indicator_Classification() :
  def __init__(self, y_test, y_pred) :
    self.y_test = y_test
    self.y_pred = y_pred

  # 분류 지표 시작
  def Precision(self) :
    return precision_score(self.y_test, self.y_pred, average = 'weighted')

  def Accuracy(self) :
    return accuracy_score(self.y_test, self.y_pred)

  def F1_score(self) :
    return f1_score(self.y_test, self.y_pred, average = 'weighted')
  
  def Recall(self) :
    return recall_score(self.y_test, self.y_pred, average = 'weighted')

  def ROC_AUC(self) : 
    y_test = self.y_test.copy()
    multi_class = 'binary'
    if y_test.nunique()[0] > 2 :
      multi_class = 'raise'

    classes = y_test.iloc[:, 0].unique()
    y_test_lb = label_binarize(self.y_test, classes = classes)
    y_pred_lb = label_binarize(self.y_pred, classes = classes)

    return roc_auc_score(y_test_lb, y_pred_lb, multi_class = multi_class)
  # 분류 지표 끝

In [8]:
class Indicator_Regression() :
  def __init__(self, y_test, y_pred) :
    self.y_test = y_test
    self.y_pred = y_pred
    
  # 회귀 지표 시작
  def MAE(self) :
    return mean_absolute_error(self.y_test, self.y_pred)

  def MSE(self) :
    return mean_squared_error(self.y_test, self.y_pred)

  def R2_score(self) :
    return r2_score(self.y_test, self.y_pred)

  def RMSE(self) :
    return math.sqrt(mean_squared_error(self.y_test, self.y_pred))
  # 회귀 지표 끝